In [44]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("YouWuA2SA")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [45]:
textEn = spark_context.textFile("hdfs://192.168.1.153:9000//europarl/europarl-v7.sv-en.en").cache()

In [46]:
textEn.count()

1862234

In [47]:
textEn.getNumPartitions()

2

In [48]:
textSv = spark_context.textFile("hdfs://192.168.1.153:9000//europarl/europarl-v7.sv-en.sv").cache()

In [49]:
textSv.count()

1862234

In [50]:
textSv.getNumPartitions()

3

In [71]:
def TokentoLowerCase(lines):
    lines = lines.lower()
    lines = lines.split()
    return lines

In [52]:
textEn1 = textEn.map(TokentoLowerCase)
textSv1 = textSv.map(TokentoLowerCase)
textEn1.take(10)

[['resumption', 'of', 'the', 'session'],
 ['i',
  'declare',
  'resumed',
  'the',
  'session',
  'of',
  'the',
  'european',
  'parliament',
  'adjourned',
  'on',
  'friday',
  '17',
  'december',
  '1999,',
  'and',
  'i',
  'would',
  'like',
  'once',
  'again',
  'to',
  'wish',
  'you',
  'a',
  'happy',
  'new',
  'year',
  'in',
  'the',
  'hope',
  'that',
  'you',
  'enjoyed',
  'a',
  'pleasant',
  'festive',
  'period.'],
 ['although,',
  'as',
  'you',
  'will',
  'have',
  'seen,',
  'the',
  'dreaded',
  "'millennium",
  "bug'",
  'failed',
  'to',
  'materialise,',
  'still',
  'the',
  'people',
  'in',
  'a',
  'number',
  'of',
  'countries',
  'suffered',
  'a',
  'series',
  'of',
  'natural',
  'disasters',
  'that',
  'truly',
  'were',
  'dreadful.'],
 ['you',
  'have',
  'requested',
  'a',
  'debate',
  'on',
  'this',
  'subject',
  'in',
  'the',
  'course',
  'of',
  'the',
  'next',
  'few',
  'days,',
  'during',
  'this',
  'part-session.'],
 ['in',
  

In [53]:
textSv1.take(10)

[['återupptagande', 'av', 'sessionen'],
 ['jag',
  'förklarar',
  'europaparlamentets',
  'session',
  'återupptagen',
  'efter',
  'avbrottet',
  'den',
  '17',
  'december.',
  'jag',
  'vill',
  'på',
  'nytt',
  'önska',
  'er',
  'ett',
  'gott',
  'nytt',
  'år',
  'och',
  'jag',
  'hoppas',
  'att',
  'ni',
  'haft',
  'en',
  'trevlig',
  'semester.'],
 ['som',
  'ni',
  'kunnat',
  'konstatera',
  'ägde',
  '"den',
  'stora',
  'år',
  '2000-buggen"',
  'aldrig',
  'rum.',
  'däremot',
  'har',
  'invånarna',
  'i',
  'ett',
  'antal',
  'av',
  'våra',
  'medlemsländer',
  'drabbats',
  'av',
  'naturkatastrofer',
  'som',
  'verkligen',
  'varit',
  'förskräckliga.'],
 ['ni',
  'har',
  'begärt',
  'en',
  'debatt',
  'i',
  'ämnet',
  'under',
  'sammanträdesperiodens',
  'kommande',
  'dagar.'],
 ['till',
  'dess',
  'vill',
  'jag',
  'att',
  'vi,',
  'som',
  'ett',
  'antal',
  'kolleger',
  'begärt,',
  'håller',
  'en',
  'tyst',
  'minut',
  'för',
  'offren',
  'f

In [54]:
display(textEn1.count())
display(textSv1.count())

1862234

1862234

In [55]:
textEn2 = textEn1.flatMap(lambda word : word).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
textEn2.takeOrdered(10, key = lambda x: -x[1])

[('the', 3498574),
 ('of', 1659884),
 ('to', 1539823),
 ('and', 1288620),
 ('in', 1086089),
 ('that', 797576),
 ('a', 773812),
 ('is', 758087),
 ('for', 534270),
 ('we', 522879)]

In [56]:
textSv2 = textSv1.flatMap(lambda word : word).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
textSv2.takeOrdered(10, key = lambda x: -x[1])

[('att', 1706309),
 ('och', 1344895),
 ('i', 1050989),
 ('det', 924878),
 ('som', 913302),
 ('för', 908703),
 ('av', 738102),
 ('är', 694389),
 ('en', 620347),
 ('vi', 539808)]

In [57]:
EnIndex = textEn1.zipWithIndex()
SvIndex = textSv1.zipWithIndex()

In [58]:
EnIndex = EnIndex.map(lambda word : (word[1], word[0]))
SvIndex = SvIndex.map(lambda word : (word[1], word[0]))

In [59]:
dic = EnIndex.join(SvIndex).sortBy(lambda word : word[0])
dic.take(1)

[(0,
  (['resumption', 'of', 'the', 'session'],
   ['återupptagande', 'av', 'sessionen']))]

In [60]:
dic1 = dic.filter(lambda word: len(word[1][0]) > 0 and len(word[1][1]) > 0)

In [61]:
dic1.count()

1848423

In [62]:
dic2 = dic1.filter(lambda word: len(word[1][1]) < 5)

In [63]:
dic2.count()

62071

In [64]:
dic3 = dic2.filter(lambda word: len(word[1][0]) == len(word[1][1]))

In [65]:
dic3.count()

26683

In [66]:
dic3.take(10)

[(50, (['agenda'], ['arbetsplan'])),
 (96, (['that', 'did', 'not', 'happen.'], ['så', 'blev', 'inte', 'fallet.'])),
 (183,
  (['the', 'debate', 'is', 'closed.'],
   ['jag', 'förklarar', 'debatten', 'avslutad.'])),
 (255,
  (['the', 'debate', 'is', 'closed.'],
   ['jag', 'förklarar', 'debatten', 'avslutad.'])),
 (468,
  (['the', 'debate', 'is', 'closed.'],
   ['jag', 'förklarar', 'debatten', 'avslutad.'])),
 (633,
  (['the', 'debate', 'is', 'closed.'],
   ['jag', 'förklarar', 'debatten', 'avslutad.'])),
 (638,
  (['are', 'there', 'any', 'comments?'],
   ['finns', 'det', 'några', 'synpunkter?'])),
 (1295, (['why?'], ['varför?'])),
 (1297, (['no.'], ['nej.'])),
 (1521,
  (['nothing', 'has', 'changed', 'there.'],
   ['det', 'har', 'inte', 'ändrats.']))]

In [67]:
dicEn = dic3.flatMap(lambda word: word[1][0])
dicSv = dic3.flatMap(lambda word: word[1][1])
dicpair = dicEn.zip(dicSv)

In [68]:
dicpair.take(10)

[('agenda', 'arbetsplan'),
 ('that', 'så'),
 ('did', 'blev'),
 ('not', 'inte'),
 ('happen.', 'fallet.'),
 ('the', 'jag'),
 ('debate', 'förklarar'),
 ('is', 'debatten'),
 ('closed.', 'avslutad.'),
 ('the', 'jag')]

In [74]:
transpair = dicpair.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
transpair = transpair.sortBy(lambda word : -word[1]).map(lambda word : word[0])
transpair.take(10)

[('closed.', 'avslutad.'),
 ('is', 'är'),
 ('(applause)', '(applåder)'),
 ('.', '.'),
 ('is', 'debatten'),
 ('the', 'jag'),
 ('debate', 'förklarar'),
 ('the', 'debatten'),
 ('is', 'härmed'),
 ('debate', 'är')]

In [70]:
# release the cores for another application!
#spark_context.stop()